In [1]:
from heapq import heappush, heappop

In [2]:
def create_cave(depth, target, x_max, y_max):
    erosion_level = {(0, 0): depth % 20183, target: depth % 20183}
    for x in range(x_max + 1):
        for y in range(y_max + 1):
            if (x, y) == target:
                continue
            if y == 0:
                erosion_level[x, y] = (x * 16807 + depth) % 20183
            elif x == 0:
                erosion_level[x, y] = (y * 48271 + depth) % 20183
            else:
                erosion_level[x, y] = (erosion_level[x, y - 1] * erosion_level[x - 1, y] + depth) % 20183
    cave = {}
    for (x, y), el in erosion_level.items():
        cave[x, y] = el % 3
    return cave

# torch = 0, climbing gear = 1, neither = 2
# rocky = 0, wet = 1, narrow = 2
def get_adjacent(cave, u):
    allowed = {0: (0, 1), 1: (1, 2), 2: (0, 2)}
    x, y, tool = u
    region = cave[x, y]
    adj = []
    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        xn, yn = x + dx, y + dy
        if xn < 0 or yn < 0:
            continue
        region_new = cave[xn, yn]
        if tool in allowed[region_new]:
            adj.append((1, (xn, yn, tool)))
    other = allowed[region][1] if tool == allowed[region][0] else allowed[region][0]
    adj.append((7, (x, y, other)))
    return adj

def dijkstra(cave, source, target):
    dist = {source: 0}
    pq = []
    heappush(pq, (dist[source], source))
    while pq:
        _, u = heappop(pq)
        if u == target:
            return dist[target]
        for d, v in get_adjacent(cave, u):
            dn = dist[u] + d
            if v not in dist or dn < dist[v]:
                dist[v] = dn
                heappush(pq, (dn, v))
    return 'Not found!'

def print_cave(cave):
    x_max = max(cave, key=lambda x: x[0])[0]
    y_max = max(cave, key=lambda x: x[1])[1]
    for y in range(y_max + 1):
        l = []
        for x in range(x_max + 1):
            match cave[x, y]:
                case 0:
                    c = '.'
                case 1:
                    c = '='
                case 2:
                    c = '|'
            l.append(c)
        print(''.join(l))

In [3]:
depth = 10647
target = (7,770)
cave = create_cave(depth, target, target[0], target[1])

In [4]:
sum(cave.values())

6208

In [5]:
cave = create_cave(depth, target, 1000, 1000)
dist = dijkstra(cave, (0, 0, 0), (*target, 0))

In [6]:
dist

1039